<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/FINAL_PROJECT_%CE%9D%CE%95%CE%A7%CE%A4_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


In [ ]:
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1


In [ ]:
import sparknlp
spark = sparknlp.start()


In [ ]:
!wget -O train_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
!wget -O test_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv

train_data = spark.read.csv("train_data.csv", inferSchema=True, header=True)
test_data = spark.read.csv("test_data.csv", inferSchema=True, header=True)


--2023-07-08 09:31:39--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14254503 (14M) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>]  13.59M  87.8MB/s    in 0.2s    

2023-07-08 09:31:39 (87.8 MB/s) - ‘train_data.csv’ saved [14254503/14254503]

--2023-07-08 09:31:39--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting resp

In [ ]:
print(train_data.columns)


['condition_label', 'medical_abstract']


In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Aggregate the BERT embeddings at the sentence level
sentence_embeddings_bert = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("bert_sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Define the classifier
classifier = ClassifierDLApproach() \
    .setInputCols(["document", "bert_sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("condition_label") \
    .setMaxEpochs(10) \
    .setEnableOutputLogs(True)

# Add the classifier to the pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    word_embeddings,
    bert_embeddings,
    sentence_embeddings_bert,
    classifier
])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Transform the training and test data
train_data = model.transform(train_data)
test_data = model.transform(test_data)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
model.write().overwrite().save("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")


In [ ]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col

# Load the trained model
model = PipelineModel.load("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")

# Check if 'bert_embeddings' column exists in the test data
if 'bert_embeddings' in test_data.columns:
    # If it exists, drop the column
    test_data = test_data.drop('bert_embeddings')

# Make predictions on the test data
predictions = model.transform(test_data)


In [ ]:
# Print the stages of the model
for stage in model.stages:
    print(stage)


DocumentAssembler_53eb81db7076
REGEX_TOKENIZER_9494d84f77a4
NORMALIZER_ab7521dd4ce4
StopWordsCleaner_08e24ff72e1b
WORD_EMBEDDINGS_MODEL_48cffc8b9a76
BERT_EMBEDDINGS_a6fae8f56338
SentenceEmbeddings_ea61ab8816d0
ClassifierDLModel_9b34e70fdfcc


In [ ]:
# Print out a few rows of the test data
test_data.show()

# Transform the test data
predictions = model.transform(test_data)

# Print out a few rows of the predictions
predictions.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|         cleanTokens|    glove_embeddings|bert_sentence_embeddings|               class|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{word_embeddings...|    [{sentence_embedd...|[{category, 0, 15...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{word_embeddings...|    [{sentence_embedd...|[{category, 0, 16...|
|              5|A phase II study ...|[{document, 

In [ ]:
from sparknlp.annotator import ClassifierDLModel

# Load the trained model
model = PipelineModel.load("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")

# Get the ClassifierDLModel stage
classifier = model.stages[-1]  # assuming the ClassifierDLModel is the last stage in the pipeline

# Print the current configuration of the ClassifierDLModel stage
print(classifier.extractParamMap())

# Now you can use the model to make predictions
predictions = model.transform(test_data)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52522)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, c

Py4JError: ignored

In [ ]:
from sparknlp.annotator import ClassifierDLModel

# Load the trained model
model = PipelineModel.load("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")

# Get the ClassifierDLModel stage
classifier = model.stages[-1]  # assuming the ClassifierDLModel is the last stage in the pipeline

# Reconfigure the ClassifierDLModel stage
classifier = ClassifierDLModel.load('/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ') \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("prediction")

# Replace the old ClassifierDLModel stage with the reconfigured one
model.stages[-1] = classifier

# Now you can use the model to make predictions
predictions = model.transform(test_data)


Py4JJavaError: ignored

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Instantiate the evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

# Compute the classification error on test data
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


IllegalArgumentException: ignored